## A very naive experiment

I was thinking what if instead of getting image embeddings what if I can just get the final output and combine it with the other attributes present here using a LGBR.

Long version:

1. I passed the images through a pretrained feature extractor (EffficientNet B7)
2. I got the final output and submitted one entry. LB (18.86)
3. I combined the output from Step 2 with the other metadata and modelled using a very naive LGB regressor.
4. On CV I got 17.33 but LB is much worse than Step 2 (18.98)
5. I want to know if this naive experiment is structered incorrectly.



In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
import os 
from sklearn import metrics, model_selection
from sklearn import preprocessing
import time
import gc
from tqdm.notebook import tqdm
import joblib
import lightgbm as lgb
import seaborn as sns

In [ ]:
gc.enable()

NUM_FOLDS = 5
SEED = 541

In [ ]:
train_df_preds = pd.concat([pd.read_csv('../input/pretrained-feature-model-keras/train_preds.csv'), pd.read_csv('../input/pretrained-feature-model-keras/valid_preds.csv')]).reset_index(drop=True)
train_df_preds.head()

In [ ]:
test_df_preds = pd.read_csv('../input/pretrained-feature-model-keras/submission.csv')
test_df_preds.head()

In [ ]:
def create_folds(data):
    data["kfold"] = -1
    data = data.sample(frac=1, random_state=SEED).reset_index(drop=True)


    kf = model_selection.KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
    for f, (t_, v_) in enumerate(kf.split(X=data)):
        data.loc[v_, 'kfold'] = f  
    
    return data

test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df = pd.concat([test_df, test_df_preds], axis=1)

train_df = train_df_preds.copy()
train_df.drop(['Id', 'path'], axis=1, inplace=True)

test_df.drop(['Id'], axis=1, inplace=True)
test_df.rename(columns={'Pawpularity':'preds'}, inplace=True)

cat_cols = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

for col in cat_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')
    
    
train_data = create_folds(train_df)

In [ ]:
scores = []

test_probs = pd.DataFrame()
train_probs = pd.DataFrame()

for fold in range(NUM_FOLDS):
    train = train_data[train_data['kfold'] != fold].reset_index(drop=True)
    val = train_data[train_data['kfold'] == fold].reset_index(drop=True)
    
    xtrain = train.drop(['Pawpularity', 'kfold'], axis=1)
    xval = val.drop(['Pawpularity', 'kfold'], axis=1)
    
    model = lgb.LGBMRegressor(n_jobs=-1)
    model.fit(xtrain, train.Pawpularity.values)
    
    save_to = 'lgb_fold{}.txt'.format(fold)
    model.booster_.save_model(save_to)
    
    predictions = model.predict(xval)
    train_probs[f'fold_{fold}'] = model.predict(train_data.drop(['Pawpularity','kfold'], axis=1))
    test_probs[f'fold_{fold}'] = model.predict(test_df)

    score = metrics.mean_squared_error(val.Pawpularity.values, predictions, squared=False)
    print(f"Fold: {fold}, Score: {score}")
    scores.append(score)
    
    
    lgb.plot_importance(model)
    plt.title(f"Feature importance fold: {fold}")
    plt.show()
    
print(f"CV: {np.mean(scores)}")

In [ ]:
ss = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
ss['Pawpularity'] = test_probs.mean(axis=1)
ss[['Id','Pawpularity']].to_csv('submission.csv', index=False)